In [28]:
"""
LangGraph Chatbot
=============================================================================

A comprehensive implementation of a conversational AI chatbot using LangGraph
with proper organization, error handling, and testing capabilities.

Author: Niyantarana Tagore
Date: 2025-06-29
"""

import os
import uuid
import logging
from typing import Dict, List, Any, Optional
from typing_extensions import TypedDict
from dataclasses import dataclass


In [29]:
!pip install -q langgraph langsmith langchain-openai python-dotenv
!pip install -q matplotlib graphviz
!pip install langgraph-checkpoint-sqlite

In [30]:
# Third-party imports
try:
    from langgraph.graph import StateGraph, START, END
    from langgraph.graph.message import add_messages
    from langgraph.checkpoint.memory import MemorySaver
    from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
    from langchain_openai import ChatOpenAI
except ImportError as e:
    raise ImportError(f"Required packages not installed: {e}")


In [32]:
# Configuration and Constants
@dataclass
class ChatbotConfig:
    """Configuration settings for the chatbot."""
    model_name: str = "gpt-4o-mini"
    temperature: float = 0.7
    max_retries: int = 3
    timeout: int = 30
    


In [33]:
class ConversationState(TypedDict):
    """Type definition for conversation state."""
    messages: List[BaseMessage]

In [34]:
# Logging setup
def setup_logging(level: int = logging.INFO) -> logging.Logger:
    """Configure logging for the application."""
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

# Initialize logger
logger = setup_logging()

In [35]:
class EnvironmentManager:
    """Manages environment setup and API key configuration."""
    
    @staticmethod
    def setup_openai_key() -> bool:
        """Setup OpenAI API key."""
        try:
            if not os.environ.get("OPENAI_API_KEY"):
                import getpass
                openai_key = getpass.getpass("Enter your OpenAI API Key: ")
                if not openai_key.strip():
                    logger.warning("No OpenAI API key provided")
                    return False
                os.environ["OPENAI_API_KEY"] = openai_key
            logger.info("✅ OpenAI API key configured")
            return True
        except Exception as e:
            logger.error(f"Failed to setup OpenAI key: {e}")
            return False
    
    @staticmethod
    def setup_langsmith_key() -> bool:
        """Setup LangSmith API key (optional)."""
        try:
            if not os.environ.get("LANGSMITH_API_KEY"):
                import getpass
                print("LangSmith setup (optional but recommended for debugging):")
                langsmith_key = getpass.getpass(
                    "Enter your LangSmith API Key (or press Enter to skip): "
                )
                
                if langsmith_key.strip():
                    os.environ["LANGSMITH_API_KEY"] = langsmith_key
                    os.environ["LANGCHAIN_TRACING_V2"] = "true"
                    os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
                    logger.info("✅ LangSmith tracing enabled")
                    return True
                else:
                    logger.info("⚠️ Skipping LangSmith setup")
                    return False
            return True
        except Exception as e:
            logger.error(f"Failed to setup LangSmith: {e}")
            return False
    
    @classmethod
    def setup_environment(cls) -> bool:
        """Setup complete environment."""
        try:
            openai_success = cls.setup_openai_key()
            cls.setup_langsmith_key()  # Optional, don't fail if it doesn't work
            
            if openai_success:
                logger.info("🚀 Environment setup complete")
                return True
            else:
                logger.error("❌ Environment setup failed - OpenAI key required")
                return False
        except Exception as e:
            logger.error(f"Environment setup failed: {e}")
            return False

In [20]:
class LLMManager:
    """Manages the Language Model instance and operations."""
    
    def __init__(self, config: ChatbotConfig):
        self.config = config
        self._llm = None
    
    @property
    def llm(self) -> ChatOpenAI:
        """Lazy initialization of the LLM."""
        if self._llm is None:
            self._llm = self._create_llm()
        return self._llm
    
    def _create_llm(self) -> ChatOpenAI:
        """Create and configure the language model."""
        try:
            llm = ChatOpenAI(
                model=self.config.model_name,
                temperature=self.config.temperature,
                max_retries=self.config.max_retries,
                request_timeout=self.config.timeout,
            )
            logger.info(f"✅ Language model initialized: {self.config.model_name}")
            return llm
        except Exception as e:
            logger.error(f"Failed to initialize LLM: {e}")
            raise
    
    def test_connection(self) -> bool:
        """Test the LLM connection."""
        try:
            response = self.llm.invoke("Say hello in a friendly way!")
            logger.info(f"✅ LLM test successful: {response.content[:50]}...")
            return True
        except Exception as e:
            logger.error(f"LLM test failed: {e}")
            return False

In [21]:
class ChatbotNode:
    """Encapsulates the chatbot node functionality."""
    
    def __init__(self, llm_manager: LLMManager):
        self.llm_manager = llm_manager
    
    def process(self, state: ConversationState) -> Dict[str, List[BaseMessage]]:
        """
        Process the conversation state and generate a response.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Dict containing new messages to add to state
            
        Raises:
            Exception: If LLM processing fails
        """
        try:
            messages = state["messages"]
            if not messages:
                raise ValueError("No messages in state")
            
            response = self.llm_manager.llm.invoke(messages)
            
            if not response or not response.content:
                raise ValueError("Empty response from LLM")
            
            return {"messages": [response]}
            
        except Exception as e:
            logger.error(f"Chatbot node processing failed: {e}")
            # Return an error message instead of crashing
            error_response = AIMessage(
                content="I apologize, but I encountered an error processing your request. Please try again."
            )
            return {"messages": [error_response]}

In [22]:
class ConversationManager:
    """Manages conversation threads and memory."""
    
    def __init__(self):
        self.memory = MemorySaver()
        self._active_threads: Dict[str, Any] = {}
    
    def create_thread_id(self) -> str:
        """Generate a unique thread ID."""
        return str(uuid.uuid4())
    
    def get_config(self, thread_id: str) -> Dict[str, Any]:
        """Get configuration for a conversation thread."""
        return {"configurable": {"thread_id": thread_id}}
    
    def list_active_threads(self) -> List[str]:
        """List all active conversation threads."""
        return list(self._active_threads.keys())


In [23]:
class LangGraphChatbot:
    """Main chatbot class that orchestrates all components."""
    
    def __init__(self, config: Optional[ChatbotConfig] = None):
        self.config = config or ChatbotConfig()
        self.llm_manager = LLMManager(self.config)
        self.conversation_manager = ConversationManager()
        self.chatbot_node = ChatbotNode(self.llm_manager)
        
        # Build graphs
        self._simple_graph = None
        self._memory_graph = None
    
    def _build_simple_graph(self):
        """Build the simple chatbot graph without memory."""
        try:
            graph_builder = StateGraph(ConversationState)
            graph_builder.add_node("chatbot", self.chatbot_node.process)
            graph_builder.add_edge(START, "chatbot")
            graph_builder.add_edge("chatbot", END)
            
            self._simple_graph = graph_builder.compile()
            logger.info("✅ Simple chatbot graph created")
            
        except Exception as e:
            logger.error(f"Failed to build simple graph: {e}")
            raise
    
    def _build_memory_graph(self):
        """Build the chatbot graph with memory."""
        try:
            graph_builder = StateGraph(ConversationState)
            graph_builder.add_node("chatbot", self.chatbot_node.process)
            graph_builder.add_edge(START, "chatbot")
            graph_builder.add_edge("chatbot", END)
            
            self._memory_graph = graph_builder.compile(
                checkpointer=self.conversation_manager.memory
            )
            logger.info("✅ Memory-enabled chatbot graph created")
            
        except Exception as e:
            logger.error(f"Failed to build memory graph: {e}")
            raise
    
    @property
    def simple_graph(self):
        """Get the simple graph (lazy initialization)."""
        if self._simple_graph is None:
            self._build_simple_graph()
        return self._simple_graph
    
    @property
    def memory_graph(self):
        """Get the memory-enabled graph (lazy initialization)."""
        if self._memory_graph is None:
            self._build_memory_graph()
        return self._memory_graph
    
    def chat_simple(self, user_input: str) -> Dict[str, Any]:
        """
        Simple chat without memory.
        
        Args:
            user_input: User's message
            
        Returns:
            Conversation result
        """
        try:
            initial_state = {
                "messages": [HumanMessage(content=user_input)]
            }
            return self.simple_graph.invoke(initial_state)
            
        except Exception as e:
            logger.error(f"Simple chat failed: {e}")
            raise
    
    def chat_with_memory(self, user_input: str, thread_id: Optional[str] = None) -> Dict[str, Any]:
        """
        Chat with memory persistence.
        
        Args:
            user_input: User's message
            thread_id: Optional thread ID (creates new if None)
            
        Returns:
            Conversation result
        """
        try:
            if thread_id is None:
                thread_id = self.conversation_manager.create_thread_id()
            
            config = self.conversation_manager.get_config(thread_id)
            
            result = self.memory_graph.invoke(
                {"messages": [HumanMessage(content=user_input)]},
                config
            )
            
            return result
            
        except Exception as e:
            logger.error(f"Memory chat failed: {e}")
            raise
    
    def initialize(self) -> bool:
        """Initialize the chatbot system."""
        try:
            # Setup environment
            if not EnvironmentManager.setup_environment():
                return False
            
            # Test LLM connection
            if not self.llm_manager.test_connection():
                return False
            
            # Build graphs (lazy initialization will happen on first use)
            logger.info("🎉 Chatbot system initialized successfully")
            return True
            
        except Exception as e:
            logger.error(f"Initialization failed: {e}")
            return False

In [24]:
class ChatbotTester:
    """Testing utilities for the chatbot system."""
    
    def __init__(self, chatbot: LangGraphChatbot):
        self.chatbot = chatbot
    
    def print_conversation(self, result: Dict[str, Any], title: str = ""):
        """Pretty print conversation results."""
        if title:
            print(f"\n🤖 {title}")
            print("=" * (len(title) + 4))
        
        messages = result.get("messages", [])
        for message in messages:
            if isinstance(message, HumanMessage):
                print(f"👤 Human: {message.content}")
            elif isinstance(message, AIMessage):
                print(f"🤖 AI: {message.content}")
        print()
    
    def test_simple_chatbot(self, test_cases: List[str]):
        """Test simple chatbot functionality."""
        print("🧪 Testing Simple Chatbot")
        print("=" * 40)
        
        for i, test_case in enumerate(test_cases, 1):
            try:
                result = self.chatbot.chat_simple(test_case)
                self.print_conversation(result, f"Test Case {i}")
                
            except Exception as e:
                logger.error(f"Test case {i} failed: {e}")
    
    def test_memory_functionality(self, thread_id: str = "test_conversation"):
        """Test memory functionality."""
        print("🧠 Testing Memory Functionality")
        print("=" * 50)
        
        test_sequence = [
            "Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.",
            "What's my name and what do I do?",
            "What programming languages should I learn for AI development?"
        ]
        
        for i, user_input in enumerate(test_sequence, 1):
            try:
                result = self.chatbot.chat_with_memory(user_input, thread_id)
                self.print_conversation(result, f"Memory Test {i}")
                
            except Exception as e:
                logger.error(f"Memory test {i} failed: {e}")
    
    def test_thread_isolation(self):
        """Test that different threads maintain separate conversations."""
        print("🔄 Testing Thread Isolation")
        print("=" * 40)
        
        # First conversation
        result1 = self.chatbot.chat_with_memory(
            "My favorite color is blue", 
            "thread_1"
        )
        self.print_conversation(result1, "Thread 1 - Setup")
        
        # Second conversation
        result2 = self.chatbot.chat_with_memory(
            "My favorite color is red", 
            "thread_2"
        )
        self.print_conversation(result2, "Thread 2 - Setup")
        
        # Test memory isolation
        result3 = self.chatbot.chat_with_memory(
            "What's my favorite color?", 
            "thread_1"
        )
        self.print_conversation(result3, "Thread 1 - Recall")
        
        result4 = self.chatbot.chat_with_memory(
            "What's my favorite color?", 
            "thread_2"
        )
        self.print_conversation(result4, "Thread 2 - Recall")

In [25]:
def main():
    """Main function demonstrating the chatbot usage."""
    # Initialize chatbot
    config = ChatbotConfig(
        model_name="gpt-4o-mini",
        temperature=0.7
    )
    
    chatbot = LangGraphChatbot(config)
    
    # Initialize system
    if not chatbot.initialize():
        logger.error("Failed to initialize chatbot system")
        return
    
    # Run tests
    tester = ChatbotTester(chatbot)
    
    # Test simple functionality
    simple_test_cases = [
        "Hello! What's your name?",
        "What is the GATE cutoff for ECE?",
        "What are the subjects in GATE ECE?"
    ]
    tester.test_simple_chatbot(simple_test_cases)
    
    # Test memory functionality
    tester.test_memory_functionality()
    
    # Test thread isolation
    tester.test_thread_isolation()
    
    print("✅ All tests completed!")


if __name__ == "__main__":
    main()

2025-06-29 10:53:30,589 - __main__ - WARNING - No OpenAI API key provided


LangSmith setup (optional but recommended for debugging):


2025-06-29 10:53:32,054 - __main__ - INFO - ⚠️ Skipping LangSmith setup
2025-06-29 10:53:32,055 - __main__ - ERROR - ❌ Environment setup failed - OpenAI key required
2025-06-29 10:53:32,066 - __main__ - ERROR - Failed to initialize chatbot system


In [26]:
"""
LangGraph Chatbot Tutorial - Refactored Version
==================================================

A comprehensive implementation of a conversational AI chatbot using LangGraph
with proper organization, error handling, and testing capabilities.

Author: Refactored from original tutorial
Date: 2025-06-29
"""

import os
import uuid
import logging
from typing import Dict, List, Any, Optional
from typing_extensions import TypedDict
from dataclasses import dataclass

# Third-party imports
try:
    from langgraph.graph import StateGraph, START, END
    from langgraph.graph.message import add_messages
    from langgraph.checkpoint.memory import MemorySaver
    from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
    from langchain_openai import ChatOpenAI
except ImportError as e:
    raise ImportError(f"Required packages not installed: {e}")


# Configuration and Constants
@dataclass
class ChatbotConfig:
    """Configuration settings for the chatbot."""
    model_name: str = "gpt-4o-mini"
    temperature: float = 0.7
    max_retries: int = 3
    timeout: int = 30
    
    
class ConversationState(TypedDict):
    """Type definition for conversation state."""
    messages: List[BaseMessage]


# Logging setup
def setup_logging(level: int = logging.INFO) -> logging.Logger:
    """Configure logging for the application."""
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)


logger = setup_logging()


class EnvironmentManager:
    """Manages environment setup and API key configuration."""
    
    @staticmethod
    def setup_openai_key(api_key: str = None) -> bool:
        """
        Setup OpenAI API key.
        
        Args:
            api_key: Optional API key to use directly (for programmatic setup)
            
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            # Check if already set
            existing_key = os.environ.get("OPENAI_API_KEY")
            if existing_key and existing_key.strip():
                logger.info("✅ OpenAI API key already configured")
                return True
            
            # Use provided key if available
            if api_key and api_key.strip():
                os.environ["OPENAI_API_KEY"] = api_key.strip()
                logger.info("✅ OpenAI API key configured programmatically")
                return True
            
            # Interactive setup
            try:
                import getpass
                print("\n🔑 OpenAI API Key Setup")
                print("Get your API key from: https://platform.openai.com/api-keys")
                openai_key = getpass.getpass("Enter your OpenAI API Key: ")
                
                if not openai_key or not openai_key.strip():
                    logger.warning("❌ No OpenAI API key provided")
                    print("💡 Tip: You can also set the OPENAI_API_KEY environment variable")
                    return False
                
                # Validate key format (basic check)
                if not openai_key.strip().startswith('sk-'):
                    logger.warning("⚠️ API key doesn't look correct (should start with 'sk-')")
                    confirm = input("Continue anyway? (y/N): ")
                    if confirm.lower() != 'y':
                        return False
                
                os.environ["OPENAI_API_KEY"] = openai_key.strip()
                logger.info("✅ OpenAI API key configured")
                return True
                
            except ImportError:
                logger.error("❌ getpass module not available")
                print("Please set OPENAI_API_KEY environment variable manually")
                return False
                
        except Exception as e:
            logger.error(f"Failed to setup OpenAI key: {e}")
            return False
    
    @staticmethod
    def setup_langsmith_key(api_key: str = None) -> bool:
        """
        Setup LangSmith API key (optional).
        
        Args:
            api_key: Optional API key to use directly
            
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            # Check if already set
            existing_key = os.environ.get("LANGSMITH_API_KEY")
            if existing_key and existing_key.strip():
                logger.info("✅ LangSmith already configured")
                return True
            
            # Use provided key if available
            if api_key and api_key.strip():
                os.environ["LANGSMITH_API_KEY"] = api_key.strip()
                os.environ["LANGCHAIN_TRACING_V2"] = "true"
                os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
                logger.info("✅ LangSmith configured programmatically")
                return True
            
            # Interactive setup
            try:
                import getpass
                print("\n📊 LangSmith Setup (Optional - for debugging and monitoring)")
                print("Get your API key from: https://smith.langchain.com/")
                langsmith_key = getpass.getpass(
                    "Enter your LangSmith API Key (or press Enter to skip): "
                )
                
                if langsmith_key and langsmith_key.strip():
                    os.environ["LANGSMITH_API_KEY"] = langsmith_key.strip()
                    os.environ["LANGCHAIN_TRACING_V2"] = "true"
                    os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
                    logger.info("✅ LangSmith tracing enabled")
                    return True
                else:
                    logger.info("⚠️ Skipping LangSmith setup")
                    return False
                    
            except ImportError:
                logger.warning("getpass not available for LangSmith setup")
                return False
                
        except Exception as e:
            logger.error(f"Failed to setup LangSmith: {e}")
            return False
    
    @classmethod
    def setup_environment(cls, openai_key: str = None, langsmith_key: str = None) -> bool:
        """
        Setup complete environment.
        
        Args:
            openai_key: Optional OpenAI API key
            langsmith_key: Optional LangSmith API key
            
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            print("🚀 Setting up environment...")
            
            # Setup OpenAI (required)
            openai_success = cls.setup_openai_key(openai_key)
            if not openai_success:
                logger.error("❌ Environment setup failed - OpenAI key required")
                print("\n💡 Solutions:")
                print("1. Get an API key from: https://platform.openai.com/api-keys")
                print("2. Set environment variable: export OPENAI_API_KEY='your-key-here'")
                print("3. Pass the key directly to setup_environment(openai_key='your-key')")
                return False
            
            # Setup LangSmith (optional)
            cls.setup_langsmith_key(langsmith_key)
            
            logger.info("🚀 Environment setup complete")
            return True
            
        except Exception as e:
            logger.error(f"Environment setup failed: {e}")
            return False
    
    @staticmethod
    def validate_environment() -> bool:
        """Validate that required environment variables are set."""
        required_vars = ["OPENAI_API_KEY"]
        missing_vars = []
        
        for var in required_vars:
            if not os.environ.get(var):
                missing_vars.append(var)
        
        if missing_vars:
            logger.error(f"Missing required environment variables: {missing_vars}")
            return False
        
        return True


class LLMManager:
    """Manages the Language Model instance and operations."""
    
    def __init__(self, config: ChatbotConfig):
        self.config = config
        self._llm = None
    
    @property
    def llm(self) -> ChatOpenAI:
        """Lazy initialization of the LLM."""
        if self._llm is None:
            self._llm = self._create_llm()
        return self._llm
    
    def _create_llm(self) -> ChatOpenAI:
        """Create and configure the language model."""
        try:
            llm = ChatOpenAI(
                model=self.config.model_name,
                temperature=self.config.temperature,
                max_retries=self.config.max_retries,
                request_timeout=self.config.timeout,
            )
            logger.info(f"✅ Language model initialized: {self.config.model_name}")
            return llm
        except Exception as e:
            logger.error(f"Failed to initialize LLM: {e}")
            raise
    
    def test_connection(self) -> bool:
        """Test the LLM connection."""
        try:
            response = self.llm.invoke("Say hello in a friendly way!")
            logger.info(f"✅ LLM test successful: {response.content[:50]}...")
            return True
        except Exception as e:
            logger.error(f"LLM test failed: {e}")
            return False


class ChatbotNode:
    """Encapsulates the chatbot node functionality."""
    
    def __init__(self, llm_manager: LLMManager):
        self.llm_manager = llm_manager
    
    def process(self, state: ConversationState) -> Dict[str, List[BaseMessage]]:
        """
        Process the conversation state and generate a response.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Dict containing new messages to add to state
            
        Raises:
            Exception: If LLM processing fails
        """
        try:
            messages = state["messages"]
            if not messages:
                raise ValueError("No messages in state")
            
            response = self.llm_manager.llm.invoke(messages)
            
            if not response or not response.content:
                raise ValueError("Empty response from LLM")
            
            return {"messages": [response]}
            
        except Exception as e:
            logger.error(f"Chatbot node processing failed: {e}")
            # Return an error message instead of crashing
            error_response = AIMessage(
                content="I apologize, but I encountered an error processing your request. Please try again."
            )
            return {"messages": [error_response]}


class ConversationManager:
    """Manages conversation threads and memory."""
    
    def __init__(self):
        self.memory = MemorySaver()
        self._active_threads: Dict[str, Any] = {}
    
    def create_thread_id(self) -> str:
        """Generate a unique thread ID."""
        return str(uuid.uuid4())
    
    def get_config(self, thread_id: str) -> Dict[str, Any]:
        """Get configuration for a conversation thread."""
        return {"configurable": {"thread_id": thread_id}}
    
    def list_active_threads(self) -> List[str]:
        """List all active conversation threads."""
        return list(self._active_threads.keys())


class LangGraphChatbot:
    """Main chatbot class that orchestrates all components."""
    
    def __init__(self, config: Optional[ChatbotConfig] = None):
        self.config = config or ChatbotConfig()
        self.llm_manager = LLMManager(self.config)
        self.conversation_manager = ConversationManager()
        self.chatbot_node = ChatbotNode(self.llm_manager)
        
        # Build graphs
        self._simple_graph = None
        self._memory_graph = None
    
    def _build_simple_graph(self):
        """Build the simple chatbot graph without memory."""
        try:
            graph_builder = StateGraph(ConversationState)
            graph_builder.add_node("chatbot", self.chatbot_node.process)
            graph_builder.add_edge(START, "chatbot")
            graph_builder.add_edge("chatbot", END)
            
            self._simple_graph = graph_builder.compile()
            logger.info("✅ Simple chatbot graph created")
            
        except Exception as e:
            logger.error(f"Failed to build simple graph: {e}")
            raise
    
    def _build_memory_graph(self):
        """Build the chatbot graph with memory."""
        try:
            graph_builder = StateGraph(ConversationState)
            graph_builder.add_node("chatbot", self.chatbot_node.process)
            graph_builder.add_edge(START, "chatbot")
            graph_builder.add_edge("chatbot", END)
            
            self._memory_graph = graph_builder.compile(
                checkpointer=self.conversation_manager.memory
            )
            logger.info("✅ Memory-enabled chatbot graph created")
            
        except Exception as e:
            logger.error(f"Failed to build memory graph: {e}")
            raise
    
    @property
    def simple_graph(self):
        """Get the simple graph (lazy initialization)."""
        if self._simple_graph is None:
            self._build_simple_graph()
        return self._simple_graph
    
    @property
    def memory_graph(self):
        """Get the memory-enabled graph (lazy initialization)."""
        if self._memory_graph is None:
            self._build_memory_graph()
        return self._memory_graph
    
    def chat_simple(self, user_input: str) -> Dict[str, Any]:
        """
        Simple chat without memory.
        
        Args:
            user_input: User's message
            
        Returns:
            Conversation result
        """
        try:
            initial_state = {
                "messages": [HumanMessage(content=user_input)]
            }
            return self.simple_graph.invoke(initial_state)
            
        except Exception as e:
            logger.error(f"Simple chat failed: {e}")
            raise
    
    def chat_with_memory(self, user_input: str, thread_id: Optional[str] = None) -> Dict[str, Any]:
        """
        Chat with memory persistence.
        
        Args:
            user_input: User's message
            thread_id: Optional thread ID (creates new if None)
            
        Returns:
            Conversation result
        """
        try:
            if thread_id is None:
                thread_id = self.conversation_manager.create_thread_id()
            
            config = self.conversation_manager.get_config(thread_id)
            
            result = self.memory_graph.invoke(
                {"messages": [HumanMessage(content=user_input)]},
                config
            )
            
            return result
            
        except Exception as e:
            logger.error(f"Memory chat failed: {e}")
            raise
    
    def initialize(self) -> bool:
        """Initialize the chatbot system."""
        try:
            # Setup environment
            if not EnvironmentManager.setup_environment():
                return False
            
            # Test LLM connection
            if not self.llm_manager.test_connection():
                return False
            
            # Build graphs (lazy initialization will happen on first use)
            logger.info("🎉 Chatbot system initialized successfully")
            return True
            
        except Exception as e:
            logger.error(f"Initialization failed: {e}")
            return False


class ChatbotTester:
    """Testing utilities for the chatbot system."""
    
    def __init__(self, chatbot: LangGraphChatbot):
        self.chatbot = chatbot
    
    def print_conversation(self, result: Dict[str, Any], title: str = ""):
        """Pretty print conversation results."""
        if title:
            print(f"\n🤖 {title}")
            print("=" * (len(title) + 4))
        
        messages = result.get("messages", [])
        for message in messages:
            if isinstance(message, HumanMessage):
                print(f"👤 Human: {message.content}")
            elif isinstance(message, AIMessage):
                print(f"🤖 AI: {message.content}")
        print()
    
    def test_simple_chatbot(self, test_cases: List[str]):
        """Test simple chatbot functionality."""
        print("🧪 Testing Simple Chatbot")
        print("=" * 40)
        
        for i, test_case in enumerate(test_cases, 1):
            try:
                result = self.chatbot.chat_simple(test_case)
                self.print_conversation(result, f"Test Case {i}")
                
            except Exception as e:
                logger.error(f"Test case {i} failed: {e}")
    
    def test_memory_functionality(self, thread_id: str = "test_conversation"):
        """Test memory functionality."""
        print("🧠 Testing Memory Functionality")
        print("=" * 50)
        
        test_sequence = [
            "Hi! My name is Niyantarana Tagore and I am an AI Agent Developer.",
            "What's my name and what do I do?",
            "What programming languages should I learn for AI development?"
        ]
        
        for i, user_input in enumerate(test_sequence, 1):
            try:
                result = self.chatbot.chat_with_memory(user_input, thread_id)
                self.print_conversation(result, f"Memory Test {i}")
                
            except Exception as e:
                logger.error(f"Memory test {i} failed: {e}")
    
    def test_thread_isolation(self):
        """Test that different threads maintain separate conversations."""
        print("🔄 Testing Thread Isolation")
        print("=" * 40)
        
        # First conversation
        result1 = self.chatbot.chat_with_memory(
            "My favorite color is blue", 
            "thread_1"
        )
        self.print_conversation(result1, "Thread 1 - Setup")
        
        # Second conversation
        result2 = self.chatbot.chat_with_memory(
            "My favorite color is red", 
            "thread_2"
        )
        self.print_conversation(result2, "Thread 2 - Setup")
        
        # Test memory isolation
        result3 = self.chatbot.chat_with_memory(
            "What's my favorite color?", 
            "thread_1"
        )
        self.print_conversation(result3, "Thread 1 - Recall")
        
        result4 = self.chatbot.chat_with_memory(
            "What's my favorite color?", 
            "thread_2"
        )
        self.print_conversation(result4, "Thread 2 - Recall")


def main():
    """Main function demonstrating the chatbot usage."""
    print("🤖 LangGraph Chatbot System")
    print("=" * 40)
    
    # Initialize chatbot
    config = ChatbotConfig(
        model_name="gpt-4o-mini",
        temperature=0.7
    )
    
    chatbot = LangGraphChatbot(config)
    
    # Initialize system
    if not chatbot.initialize():
        logger.error("Failed to initialize chatbot system")
        print("\n🔧 Troubleshooting Steps:")
        print("1. Make sure you have a valid OpenAI API key")
        print("2. Check your internet connection")
        print("3. Verify all required packages are installed")
        return
    
    # Run tests
    tester = ChatbotTester(chatbot)
    
    try:
        # Test simple functionality
        print("\n" + "="*50)
        simple_test_cases = [
            "Hello! What's your name?",
            "What is the GATE cutoff for ECE?",
            "What are the subjects in GATE ECE?"
        ]
        tester.test_simple_chatbot(simple_test_cases)
        
        # Test memory functionality
        print("\n" + "="*50)
        tester.test_memory_functionality()
        
        # Test thread isolation
        print("\n" + "="*50)
        tester.test_thread_isolation()
        
        print("✅ All tests completed successfully!")
        
    except Exception as e:
        logger.error(f"Testing failed: {e}")
        print("❌ Some tests failed. Check the logs above for details.")


# Convenience function for easy setup
def quick_setup(openai_key: str = None):
    """
    Quick setup function for easy initialization.
    
    Args:
        openai_key: Your OpenAI API key
        
    Returns:
        LangGraphChatbot: Initialized chatbot instance
    """
    # Setup environment
    if not EnvironmentManager.setup_environment(openai_key=openai_key):
        return None
    
    # Create and initialize chatbot
    config = ChatbotConfig()
    chatbot = LangGraphChatbot(config)
    
    if chatbot.initialize():
        print("✅ Chatbot ready to use!")
        return chatbot
    else:
        print("❌ Failed to initialize chatbot")
        return None


# Example usage functions
def example_simple_chat():
    """Example of simple chat usage."""
    chatbot = quick_setup()
    if not chatbot:
        return
    
    # Simple chat examples
    test_messages = [
        "Hello, how are you?",
        "What can you help me with?",
        "Tell me a joke"
    ]
    
    for msg in test_messages:
        print(f"\n👤 User: {msg}")
        try:
            result = chatbot.chat_simple(msg)
            ai_response = result["messages"][-1].content
            print(f"🤖 AI: {ai_response}")
        except Exception as e:
            print(f"❌ Error: {e}")


def example_memory_chat():
    """Example of memory-enabled chat."""
    chatbot = quick_setup()
    if not chatbot:
        return
    
    thread_id = "example_conversation"
    
    # Conversation with memory
    conversation = [
        "Hi, my name is Alice and I'm a software engineer.",
        "What's my name?",
        "What do I do for work?",
        "Can you remember what we talked about?"
    ]
    
    for msg in conversation:
        print(f"\n👤 User: {msg}")
        try:
            result = chatbot.chat_with_memory(msg, thread_id)
            ai_response = result["messages"][-1].content
            print(f"🤖 AI: {ai_response}")
        except Exception as e:
            print(f"❌ Error: {e}")


if __name__ == "__main__":
    main()


# Installation requirements (run these in separate cells in Jupyter)
"""
!pip install -q langgraph langsmith langchain-openai python-dotenv
!pip install -q matplotlib graphviz
!pip install langgraph-checkpoint-sqlite
"""

🤖 LangGraph Chatbot System
🚀 Setting up environment...

🔑 OpenAI API Key Setup
Get your API key from: https://platform.openai.com/api-keys


2025-06-29 10:53:33,768 - __main__ - WARNING - ❌ No OpenAI API key provided
2025-06-29 10:53:33,769 - __main__ - ERROR - ❌ Environment setup failed - OpenAI key required
2025-06-29 10:53:33,776 - __main__ - ERROR - Failed to initialize chatbot system


💡 Tip: You can also set the OPENAI_API_KEY environment variable

💡 Solutions:
1. Get an API key from: https://platform.openai.com/api-keys
2. Set environment variable: export OPENAI_API_KEY='your-key-here'
3. Pass the key directly to setup_environment(openai_key='your-key')

🔧 Troubleshooting Steps:
1. Make sure you have a valid OpenAI API key
2. Check your internet connection
3. Verify all required packages are installed


'\n!pip install -q langgraph langsmith langchain-openai python-dotenv\n!pip install -q matplotlib graphviz\n!pip install langgraph-checkpoint-sqlite\n'